In [ ]:
def rgb2hsv(color):
    color_list=color.tolist()
    b,g,r=color_list[0],color_list[1],color_list[2]
    r, g, b = r/255.0, g/255.0, b/255.0
    mx, mn = max(r, g, b), min(r, g, b)
    m = mx-mn
    if mx == mn:
        h = 0
    elif mx == r:
        if g >= b:
            h = ((g-b)/m)*60
        else:
            h = ((g-b)/m)*60 + 360
    elif mx == g:
        h = ((b-r)/m)*60 + 120
    elif mx == b:
        h = ((r-g)/m)*60 + 240
    if mx == 0:
        s = 0
    else:
        s = m/mx
    v = mx
    return h, s, v


丟一些測試的代碼

In [7]:
##資料庫
import pandas as pd
import numpy as np

title={
    "id":[],
    'name_En':[],
    'name_Ch':[],
    'color_hist':[]
}

flower_db=pd.DataFrame(title)


In [8]:
flower_db


,id,name_En,name_Ch,color_hist


In [12]:
import flower



111 124
